# **Generative Search**

In [1]:
# Install packages
!pip install -q -U sentence-transformers
!pip install -q torch==2.1.0
!pip install -q -U transformers
!pip install -q accelerate
!pip install -q -U langchain


# Import packages
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import pandas as pd
import numpy as np
import json
from sentence_transformers import SentenceTransformer
from transformers import pipeline
import transformers
import torch
from langchain import PromptTemplate, LLMChain
from langchain.llms import HuggingFacePipeline
from sklearn.metrics.pairwise import cosine_similarity
import warnings

warnings.filterwarnings('ignore')


# Load Models
# Load LLM
generate_text = pipeline(
    model="databricks/dolly-v2-3b",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    temperature=0.25,
    max_time=30,
    device_map="auto",
    return_full_text=True)

# template for an instruction with input
prompt_with_context = PromptTemplate(
    input_variables=["instruction", "context"],
    template="{instruction}\n\nInput:\n{context}")

# Load HuggingFacePipeline
hf_pipeline = HuggingFacePipeline(pipeline=generate_text)

# Define LLMChain
llm_context_chain = LLMChain(llm=hf_pipeline, prompt=prompt_with_context)

# Load Bert sentence transformer
model_minilm = SentenceTransformer('all-MiniLM-L6-v2')


# Read Embedded Q&A dataset
# Mount the files in drive folder
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Function: Read dataset
def read_data():
  df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/TIS_Embedded_Q&A.csv')

  # Convert the string embeddings back to the original format
  df['embedded_questions'] =  df['embedded_questions'].apply(lambda x: np.array(json.loads(x), dtype=np.float32))

  # Convert embedded_question ad reponse to arrays
  embedded_questions = np.array(df['embedded_questions'].to_list())
  responses = np.array(df['response'].to_list())

  return df, embedded_questions, responses

df, embedded_questions, responses = read_data()


# Online Generative Search
# Function: Embed input text
def embed_input_text(input_text):
    return model_minilm.encode([input_text])


# Function: Cosine similarity check to find top_response
def find_top_response(input_text_embedded):
  # Compute Cosine similarity
    cosine_score_ls = cosine_similarity(input_text_embedded, embedded_questions)

    # Flatten the list and filter on similarity score above 0.7
    similar_indices = [i for i, score in enumerate(cosine_score_ls[0]) if score > 0.7]

    if similar_indices:
        # Find the index of the top similarity score
        top_index = max(similar_indices, key=lambda i: cosine_score_ls[0][i])

        # Return the corresponding response
        return responses[top_index]
    else:
        return "No response found"


# Function: Generative search
def generative_search(input_text):

    # Embed input text
    input_text_embedded = embed_input_text(input_text)

    # Compute Cosine similarity to find top_response
    top_response = find_top_response(input_text_embedded)

    if top_response == "No response found":
        return "I'm sorry, but I don't have expertise in this topic; my training data is limited to only “CS 410 Text Information Systems”."
    else:
      return llm_context_chain.predict(instruction=input_text, context=top_response).lstrip()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.5/177.5 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.0/47.0 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.0 MB/s eta 0:00:00


config.json:   0%|          | 0.00/819 [00:00<?, ?B/s]

instruct_pipeline.py:   0%|          | 0.00/9.16k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/databricks/dolly-v2-3b:
- instruct_pipeline.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


pytorch_model.bin:   0%|          | 0.00/5.68G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

Mounted at /content/drive


In [4]:
# Run Generative Search
print('Enter your input question about "CS 410 Text Information Systems":')
generative_search(input())

Enter your input question about "CS 410 Text Information Systems":
Can you explain how do random variables play a role in the understanding of queries and documents?


'Random variables are used to model the uncertainty in the content and intent of queries and documents.'